In [1]:
import fitz  # PyMuPDF


def extract_text_pymupdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text("text") + "\n"
    return text


pdf_text = extract_text_pymupdf("1659341275-.pdf")

chunk_size = 200
chunks = [pdf_text[i:i + chunk_size] for i in range(0, len(pdf_text), chunk_size)]  # تقسیم به chunks

In [2]:
from sentence_transformers import SentenceTransformer  # SentenceTransformer.shape() --> مثلاً ۳۸۴ یا ۷۶۸ بعدی
import numpy as np

# مدل چندزبانه MiniLM که فارسی رو هم پشتیبانی می‌کنه
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# گرفتن embedding برای هر chunk
embeddings = embedder.encode(chunks, convert_to_tensor=False)
embeddings = np.array(embeddings).astype("float32")  # خروجی تبدیل به float32 میشه چون FAISS فقط با float32 کار می‌کنه.

# اینجا embeddingها رو داخل یک دیتابیس سریع جستجو قرار می‌دیم:
import faiss

dimension = embeddings.shape[1]  # تعداد ابعاد بردار (مثلاً 384)
index = faiss.IndexFlatL2(dimension)  # یعنی جستجو بر اساس فاصله اقلیدسی L2
index.add(embeddings)


In [3]:
# جستجوی chunkهای مرتبط

def retrive_relevant_chunks(query, top_k=3):
    query_embedding = embedder.encode([query], convert_to_tensor=False).astype("float32")
    distances, indexes = index.search(query_embedding, top_k)
    return [chunks[i] for i in indexes[0]]

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# نصف دقت (half precision) استفاده می‌کنه → سرعت بیشتر و مصرف RAM کمتر.
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", dtype=torch.float16)


In [5]:
def generate_response(query, top_k=3, max_new_tokens=150):
    relevant_chunks = retrive_relevant_chunks(query, top_k=top_k)
    context = "\n".join(relevant_chunks)

    # ساخت پرامپت
    #سؤال + context از PDF به مدل داده میشه.
    prompt = f"سؤال: {query}\n\nمتن مرتبط از سند:\n{context}\n\n"

    # آماده‌سازی ورودی
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # تولید جواب
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature=0.7,
        repetition_penalty=1.1,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip()


In [6]:
while True:
    user_query = input("سؤال شما (برای خروج بنویسید 'خروج'): ")
    if user_query.strip().lower() == "خروج":
        break
    response = generate_response(user_query)
    print("چت‌بات:", response)
